In [1]:
import open3d as o3d
eagle = o3d.data.EaglePointCloud()
pcd = o3d.io.read_point_cloud(eagle.path)

print(pcd)
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/EaglePointCloud.ply
[Open3D INFO] Downloaded to /home/nero/open3d_data/download/EaglePointCloud/EaglePointCloud.ply
PointCloud with 796825 points.


In [2]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 796825 / 368254
[Open3D DEBUG] #   Got kernel density: 0.09461307525634766 (s), 433.76171875 (MB) / 433.76171875 (MB) / 433 (MB)
[Open3D DEBUG] #     Got normal field: 0.6564919948577881 (s), 539.109375 (MB) / 539.109375 (MB) / 539 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.623549e-06 / 2.090510e+00
[Open3D DEBUG] #       Finalized tree: 0.6026420593261719 (s), 672.39453125 (MB) / 672.39453125 (MB) / 672 (MB)
[Open3D DEBUG] #  Set FEM constraints: 1.0684208869934082 (s), 619.70703125 (MB) / 672.39453125 (MB) / 672 (MB)
[Open3D DEBUG] #Set point constraints: 0.18422317504882812 (s), 619.70703125 (MB) / 672.39453125 (MB) / 672 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 2945433 / 3365000 / 1209
[Open3D DEBUG] Memory Usage: 619.707 MB
[Open3D DEBUG] # Linear system solved: 1.6434459686279297 (s), 677.48828125 (MB) / 677.48828125 (MB) / 677 (MB)
[Open3D DEBUG] Got average: 0.051601886749267

In [5]:
import pandas as pd

df_person = pd.read_csv(person_filename)
df_person

,x,y,z,CosAngle,ObjIdx,ObjTag
0,4.051992,4.033520,-0.523512,0.440928,580.0,4.0
1,4.065469,3.997915,-0.522097,0.559707,580.0,4.0
2,4.091640,3.974932,-0.522343,0.539188,580.0,4.0
3,4.133047,3.966479,-0.524532,0.355588,580.0,4.0
4,4.004062,4.034707,-0.563076,0.510506,580.0,4.0
...,...,...,...,...,...,...
531,3.914726,3.896919,-2.355879,0.784149,580.0,4.0
532,3.929805,3.864521,-2.350728,0.917807,580.0,4.0
533,3.953555,3.840776,-2.350902,0.913410,580.0,4.0
534,3.986328,3.825664,-2.356476,0.768629,580.0,4.0


In [8]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df_person.iloc[:,:3].to_numpy()) # Need to pass numpy array

In [9]:
print(pcd)
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])


PointCloud with 536 points.


In [13]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

In [22]:
print(pcd)
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101], point_show_normal=True)


PointCloud with 536 points.


In [26]:
import numpy as np
import matplotlib.pyplot as plt

print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

visualize densities


In [22]:
help(o3d.geometry.TriangleMesh.create_from_point_cloud_poisson)

Help on built-in function create_from_point_cloud_poisson in module open3d.cuda.pybind.geometry:

create_from_point_cloud_poisson(...) method of builtins.PyCapsule instance
    create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False, n_threads=-1)
    Function that computes a triangle mesh from a oriented PointCloud pcd. This implements the Screened Poisson Reconstruction proposed in Kazhdan and Hoppe, "Screened Poisson Surface Reconstruction", 2013. This function uses the original implementation by Kazhdan. See https://github.com/mkazhdan/PoissonRecon
    
    Args:
        pcd (open3d.geometry.PointCloud): PointCloud from which the TriangleMesh surface is reconstructed. Has to contain normals.
        depth (int, optional, default=8): Maximum depth of the tree that will be used for surface reconstruction. Running at depth d corresponds to solving on a grid whose resolution is no larger than 2^d x 2^d x 2^d. Note that since the reconstructor adapts the octre

In [28]:
import pandas as pd
import open3d as o3d
import numpy as np


# load point cloud
filename = "~/Documents/person_with_normal.csv"
df_person = pd.read_csv(filename)
df_person

,//X,Y,Z,curvature,Nx,Ny,Nz
0,4.051992,4.033520,-0.523512,0.054107,-0.404877,-0.772356,0.489430
1,4.065469,3.997915,-0.522097,0.057785,-0.369796,-0.802025,0.469049
2,4.091640,3.974931,-0.522343,0.043356,-0.323679,-0.816304,0.478414
3,4.133047,3.966479,-0.524532,0.037665,-0.285780,-0.826108,0.485670
4,4.004062,4.034707,-0.563076,0.055416,-0.454142,-0.734885,0.503685
...,...,...,...,...,...,...,...
531,3.914726,3.896919,-2.355879,0.070745,-0.595029,-0.178052,0.783734
532,3.929805,3.864521,-2.350729,0.095557,-0.657278,-0.241160,0.714022
533,3.953555,3.840776,-2.350902,0.103098,-0.685344,-0.254828,0.682178
534,3.986328,3.825664,-2.356476,0.110364,-0.687658,-0.213626,0.693895


In [30]:

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df_person.iloc[:, :3].to_numpy())

In [31]:
pcd.normals = o3d.utility.Vector3dVector(df_person.iloc[:, 4:].to_numpy())

In [46]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
# o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 536 / 536
[Open3D DEBUG] #   Got kernel density: 0.0061719417572021484 (s), 426.2578125 (MB) / 426.2578125 (MB) / 427 (MB)
[Open3D DEBUG] #     Got normal field: 0.014261007308959961 (s), 426.2578125 (MB) / 426.2578125 (MB) / 427 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 8.935911e-04 / 4.789648e-01
[Open3D DEBUG] #       Finalized tree: 0.03046417236328125 (s), 426.2578125 (MB) / 426.2578125 (MB) / 427 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.0330810546875 (s), 426.2578125 (MB) / 426.2578125 (MB) / 427 (MB)
[Open3D DEBUG] #Set point constraints: 0.00173187255859375 (s), 426.2578125 (MB) / 426.2578125 (MB) / 427 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 89111 / 71848 / 29993
[Open3D DEBUG] Memory Usage: 426.258 MB
[Open3D DEBUG] # Linear system solved: 0.06240105628967285 (s), 426.2578125 (MB) / 426.2578125 (MB) / 427 (MB)
[Open3D DEBUG] Got average: 0.0048980712890625 (s), 426.257

In [47]:
print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
# o3d.visualization.draw_geometries([density_mesh])



visualize densities


In [36]:
densities.max()

5.388441562652588

In [37]:
densities.min()

1.368121862411499

In [48]:
print('remove low density vertices')
vertices_to_remove = densities < np.quantile(densities, 0.5)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)

mesh.compute_vertex_normals()
# Paint it gray. Not necessary but the reflection of lighting is hardly perceivable with black surfaces.
mesh.paint_uniform_color(np.array([[0.5],[0.5],[0.5]]))

o3d.visualization.draw_geometries([mesh],)

remove low density vertices
TriangleMesh with 1489 points and 2787 triangles.


In [49]:
pcd = mesh.sample_points_uniformly(number_of_points=10000)
o3d.visualization.draw_geometries([pcd])

In [50]:
print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd],)

Downsample the point cloud with a voxel of 0.05


In [52]:
o3d.io.write_point_cloud("csv/reconstructed_person_pcd.ply", pcd)

True

In [55]:
o3d.io.write_triangle_mesh("csv/reconstructed_person_mesh.obj", mesh)

[Open3D WARNING] Write OBJ can not include triangle normals.


True

In [56]:
# loading the saved traingle mesh

In [57]:
import open3d as o3d

# Specify the file path of the mesh you want to load
file_path = "csv/reconstructed_person_mesh.obj"

# Read the triangle mesh from the specified file
mesh = o3d.io.read_triangle_mesh(file_path)

# Visualize the loaded mesh
o3d.visualization.draw_geometries([mesh])

In [62]:
# Create a scene and add the traingle mesh of the object
mesh_new = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh_new)


0

In [63]:
# A single ray that starts with 0, 0, 0 and goes to 4, 4, -1.5  ==> These end direction of the ray is taken by point picking from cloudcompare

rays = o3d.core.Tensor([[0, 0, 0, 4, 4, -1.5]],
                       dtype=o3d.core.Dtype.Float32)

ans = scene.cast_rays(rays)

In [64]:
print(ans.keys())

dict_keys(['primitive_uvs', 'primitive_ids', 'geometry_ids', 'primitive_normals', 't_hit'])


### The result contains information about a possible intersection with the geometry in the scene.

In [65]:
ans["t_hit"]

[0.99383795]
Tensor[shape={1}, stride={1}, Float32, CPU:0, 0x4cded60]

In [66]:
ans

{'primitive_uvs': [[0.1606666 0.29831153]]
 Tensor[shape={1, 2}, stride={2, 1}, Float32, CPU:0, 0x5ff07d0],
 'primitive_ids': [2725]
 Tensor[shape={1}, stride={1}, UInt32, CPU:0, 0x4cdf1c0],
 'geometry_ids': [0]
 Tensor[shape={1}, stride={1}, UInt32, CPU:0, 0x3ef6250],
 'primitive_normals': [[-0.9394242 -0.22336565 -0.25998056]]
 Tensor[shape={1, 3}, stride={3, 1}, Float32, CPU:0, 0x5784480],
 't_hit': [0.99383795]
 Tensor[shape={1}, stride={1}, Float32, CPU:0, 0x4cded60]}

In [71]:
ans["primitive_ids"]

[2725]
Tensor[shape={1}, stride={1}, UInt32, CPU:0, 0x4cdf1c0]

In [78]:
triangles = np.asarray(mesh.triangles)

In [79]:
triangles

array([[   0,    1,    2],
       [   3,    4,    2],
       [   3,    2,    1],
       ...,
       [ 826, 1481,  791],
       [1481, 1479,  791],
       [1479,  761,  791]], dtype=int32)

In [80]:
triangles[2725]

array([1452, 1454,  570], dtype=int32)

In [81]:
mesh

TriangleMesh with 1489 points and 2787 triangles.

In [82]:
len(triangles)

2787

In [83]:
vertices = np.asarray(mesh.vertices)

In [84]:
vertices

array([[ 3.95217991,  3.83322001, -2.37374997],
       [ 3.92477989,  3.86509991, -2.37374997],
       [ 3.9214201 ,  3.86509991, -2.38152003],
       ...,
       [ 4.11267996,  3.80135012, -1.49704003],
       [ 3.9533    ,  4.05635023, -1.48394001],
       [ 3.9533    ,  4.08823013, -1.50970006]])

In [85]:
len(vertices)

1489

In [87]:
v1 = vertices[1452]
v2 = vertices[1454]
v3 = vertices[500]

In [88]:
print(v1, v2, v3)

[ 3.9762001   3.96072006 -1.48125005] [ 3.96862006  3.99259996 -1.48125005] [ 4.16793013  3.73760009 -1.57686996]


In [89]:
intersected_point = [(v1[i] + v2[i] + v3[i])/3 for i in range(3)]
intersected_point

[4.037583430608113, 3.8969733715057373, -1.513123353322347]

# Till this point we are able to create a ray , raycast and get the point of intersection

## Now we will create a group of rays and perform raycasting

In [104]:
df_road = pd.read_csv("/home/nero/Documents/only_road_cloud_23k.csv", skiprows=[1,])

In [105]:
df_road

,//X,Y,Z,ObjTag
0,21.173100,11.81200,-2.40167,7.0
1,21.244699,11.68270,-2.40167,7.0
2,21.315500,11.55310,-2.40167,7.0
3,21.385500,11.42290,-2.40167,7.0
4,19.408800,11.45680,-2.40167,7.0
...,...,...,...,...
23062,-3.830350,-3.78967,-2.39327,8.0
23063,-3.855980,-3.76879,-2.39488,8.0
23064,-3.881600,-3.74789,-2.39659,8.0
23065,-3.907320,-3.72697,-2.39838,8.0


In [106]:
df_ray_origin = pd.DataFrame(np.zeros((df_road.shape[0], 3)))
df_ray_origin

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
23062,0.0,0.0,0.0
23063,0.0,0.0,0.0
23064,0.0,0.0,0.0
23065,0.0,0.0,0.0


In [107]:
df_ray_origin_with_target_direction = pd.concat([df_ray_origin, df_road.iloc[:, :3]], axis=1)
df_ray_origin_with_target_direction

,0,1,2,//X,Y,Z
0,0.0,0.0,0.0,21.173100,11.81200,-2.40167
1,0.0,0.0,0.0,21.244699,11.68270,-2.40167
2,0.0,0.0,0.0,21.315500,11.55310,-2.40167
3,0.0,0.0,0.0,21.385500,11.42290,-2.40167
4,0.0,0.0,0.0,19.408800,11.45680,-2.40167
...,...,...,...,...,...,...
23062,0.0,0.0,0.0,-3.830350,-3.78967,-2.39327
23063,0.0,0.0,0.0,-3.855980,-3.76879,-2.39488
23064,0.0,0.0,0.0,-3.881600,-3.74789,-2.39659
23065,0.0,0.0,0.0,-3.907320,-3.72697,-2.39838


In [108]:
rays_array = df_ray_origin_with_target_direction.to_numpy()
rays_array

array([[ 0.        ,  0.        ,  0.        , 21.17309952, 11.81200027,
        -2.40166998],
       [ 0.        ,  0.        ,  0.        , 21.24469948, 11.68270016,
        -2.40166998],
       [ 0.        ,  0.        ,  0.        , 21.31550026, 11.55309963,
        -2.40166998],
       ...,
       [ 0.        ,  0.        ,  0.        , -3.8815999 , -3.74789   ,
        -2.39658999],
       [ 0.        ,  0.        ,  0.        , -3.90732002, -3.72696996,
        -2.39838004],
       [ 0.        ,  0.        ,  0.        , -3.93306994, -3.70600009,
        -2.40026999]])

## We have now a group of rays, we need to figure out the intersection point on the person mesh

In [109]:
rays = o3d.core.Tensor(rays_array,dtype=o3d.core.Dtype.Float32)

ans = scene.cast_rays(rays)

In [113]:
ans.keys()

dict_keys(['primitive_uvs', 'primitive_ids', 'geometry_ids', 'primitive_normals', 't_hit'])

In [120]:
triangle_ids = [i for i in ans["primitive_ids"] if i != scene.INVALID_ID]
triangle_ids[0]

2781
Tensor[shape={}, stride={}, UInt32, CPU:0, 0x597c19c]

### These are the index of traingle with which the ray intersect.

In [118]:
mesh_triangles = mesh.triangles
mesh_vertices = mesh.vertices

In [121]:
mesh_triangles[2781]

array([1478, 1397, 1394], dtype=int32)

### We get the vetices of the triangle

## Using the index of the vertices of the intersected traingle, we can get the corresponding vertices pointvalue by getting the vertices value from mesh.vertices

## We can get the intersecting point of a single ray by taking the centroid of the intersected triangle vetices

In [134]:
v1 = mesh_vertices[mesh_triangles[2781][0]]
v2 = mesh_vertices[mesh_triangles[2781][1]]
v3 = mesh_vertices[mesh_triangles[2781][2]]

intersected_point = [(v1[i] + v2[i] + v3[i])/3 for i in range(3)]
intersected_point

[3.9378832976023355, 4.268853346506755, -1.3643699884414673]

# The above gives the centroid of triangle whose index is 2781. We got this index of triangle that intersect with the rays from raycasting => primitive_ids after raycasting to the scene